### **MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli 2 LABELS CON EXTRACCION DE ENUNCIADOS CON KEYWORD**

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install datasets
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DebertaV2ForSequenceClassification
from sklearn.metrics import classification_report
import torch
from sklearn.model_selection import train_test_split
import spacy
from google.colab import drive
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 33.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Resta

In [ ]:
# Monta Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    if found_sentences:
        return '. '.join(found_sentences)
    else:
        return text


In [ ]:
# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")


In [ ]:
# Cargar el modelo reentrenado y el tokenizador
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = DebertaV2ForSequenceClassification.from_pretrained('/content/drive/MyDrive/FINE_FINAL/checkpoint-1200', num_labels=2)

# Cargar los nuevos datos de validación
new_data = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0, 1, 2, 3], engine='python')
print(new_data.info)

<bound method DataFrame.info of           id              keyword  \
0    fc6l72u               cruise   
1     8eijpy  runners, run , run    
2    didnpbe       running , golf   
3    dxe6gbb                  sea   
4    f5jqgbz                waves   
..       ...                  ...   
595  edvs552                 walk   
596   ee31pf              outside   
597  eei3pz3        outside, walk   
598  eek8bpk              outside   
599  eeljxq0              outside   

                                                  text  label  
0    (1) I had SA but managed to practically eradic...      0  
1     Anyone looking for a friend? Real friend, beh...      0  
2     Breathe, darling. The purpose of dating is to...      0  
3     I've been overweight since 2nd grade and it a...      0  
4     I need this terribly. I’ve been in negative w...      0  
..                                                 ...    ...  
595   The thought of applying for a job terrified m...      2  
596   No on

In [ ]:
# Preprocesar los datos del nuevo archivo
new_data['text'] = new_data.apply(lambda row: extract_keyword_sentences(row), axis=1)
new_data['label'] = new_data['label'].apply(lambda x: 1 if x != 0 else 0)


In [ ]:
# Función para obtener las predicciones e imprimir el texto y la clase
def obtener_predicciones(text):
    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # Pasar los tokens al modelo
    outputs = model(**inputs)
    # Obtener la predicción
    logits = outputs.logits
    predicted_class = logits.argmax().item()
    # Imprimir el texto y la clase asignada
    print(f"Texto: {text}")
    print(f"Clase asignada: {predicted_class}")
    return predicted_class


In [ ]:
# Aplicar la función a cada texto en los datos de prueba
predictions = []
for text in new_data['text']:
    predictions.append(obtener_predicciones(text))

# Crear un DataFrame con las predicciones
results_df = pd.DataFrame({'id': new_data['id'], 'text': new_data['text'], 'label': predictions})

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Texto: Yesterday I went on a party cruise alone and made friends.
Clase asignada: 0
Texto: I shouldn't have to say it on this sub, but I'm not looking for hit-and-runners, I'm sick of them.. I run a game server and I really love doing so.. Always tried to run one when I could.
Clase asignada: 0
Texto: If you want a long-running relationship, it will come up at some point anyway.. mini golf?).
Clase asignada: 0
Texto: I kind of accidentally psyched myself out of thinking I was sea-bottom scum.
Clase asignada: 0
Texto: I’ve been in negative waves lately.
Clase asignada: 0
Texto: nnThe reality of our landscape does not change, our future is not very bright, if I am being sincere.
Clase asignada: 0
Texto: Sorry if I'm sounding too crazy, I was just having a stream of thoughts after reading your post.
Clase asignada: 0
Texto: Because I've been "forced" (by the nature of my job) to go to different people and fix their computers and what not and to answer phone calls etc.
Clase asignada: 0
Te

In [ ]:
# Contar las clases en la columna 'label'
class_counts = results_df['label'].value_counts()
y_pred = results_df['label']

# Mostrar los conteos de clases
print(class_counts)


label
0    363
1    237
Name: count, dtype: int64


In [ ]:
original_data_0_1 = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0, 1, 2, 3], engine='python')
original_data_0_1['label'] = original_data_0_1['label'].apply(lambda x: 1 if x != 0 else 0)
y_true = original_data_0_1['label']

print(classification_report(y_true, y_pred))

# Crear un DataFrame con las predicciones
results_df_original_data_0_1 = pd.DataFrame({'id': original_data_0_1['id'], 'keyword': original_data_0_1['keyword'], 'text': original_data_0_1['text'], 'label': predictions})

              precision    recall  f1-score   support

           0       0.98      0.95      0.96       377
           1       0.92      0.97      0.94       223

    accuracy                           0.96       600
   macro avg       0.95      0.96      0.95       600
weighted avg       0.96      0.96      0.96       600



In [ ]:
print(results_df_original_data_0_1.iloc[19])  # Se usa 18 ya que Python comienza a contar desde 0


id                                                    9ujfy8
keyword                     running , running , bike, riding
text        I thought I'd overcome my social anxiety but ...
label                                                      1
Name: 19, dtype: object




---

SENTIMENT ANALYSIS

https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment?text=%26gt%3B+I+used+to+have+very+bad+social+anxiety+at+the+ages+of+about+12-16+to+the+point+**i+was+scared+to+leave+the+house**+or+to+talk+to+anybody+but+close+friends+and+family.++nn++%5BAgoraphobia%5D%28https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FAgoraphobia%29+is+an+anxiety+disorder+characterized+by+symptoms+of+anxiety+in+situations+where+the+person+perceives+their+environment+to+be+unsafe+with+no+easy+way+to+escape.+These+situations+can+include+open+spaces%2C+public+transit%2C+shopping+centers%2C+**or+simply+being+outside+their+home**.%22.nn%26gt%3B+i+have+been+having+lots+of+panic+attacks+and+crying+almost+daily+over+stupid+irrational+things.+nnThat+is+kind+of+extreme.+nn%26gt%3B++I+am+constantly+worrying+about+what+people+think+of+me+and+how+i+am+coming+across%2C+i+have+a+strong+desire+to+be+liked+by+everyone+and+can+not+handle+it+when+i+am+not.+nnIt+helps+to+think+of+those+kinds+of+thoughts+as+childish.


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer


In [ ]:
# Cargar el dataframe resultante del primer código
# Filtrar el DataFrame para obtener solo las filas con etiqueta 1
filtered_data = results_df_original_data_0_1[results_df_original_data_0_1['label'] == 1]

print(filtered_data)

          id                               keyword  \
19    9ujfy8      running , running , bike, riding   
65    ppk7o2                      nature, outside    
66   d9lzwq4  beach,  sand,  sand,  breeze,  waves   
69    rbybrj                         walk,  stream   
76    eqxslg                                 climb   
..       ...                                   ...   
595  edvs552                                  walk   
596   ee31pf                               outside   
597  eei3pz3                         outside, walk   
598  eek8bpk                               outside   
599  eeljxq0                               outside   

                                                  text  label  
19    I thought I'd overcome my social anxiety but ...      1  
65    My nightmares aren’t about monsters, they are...      1  
66    First I try visualization: I try visualising ...      1  
69    cried in in front of everyone in my college c...      1  
76   Booking a rock climbing le

In [ ]:
print(filtered_data.iloc[0])

id                                                    9ujfy8
keyword                     running , running , bike, riding
text        I thought I'd overcome my social anxiety but ...
label                                                      1
Name: 19, dtype: object


In [ ]:
filtered_data['text'] = filtered_data.apply(lambda row: extract_keyword_sentences(row), axis=1)

<ipython-input-71-ce0ff409cc25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['text'] = filtered_data.apply(lambda row: extract_keyword_sentences(row), axis=1)


In [ ]:
# Obtener los textos filtrados
texts_to_classify = filtered_data['text'].tolist()

In [ ]:
# Crear el clasificador de sentimiento
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
sentiment_task = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Realizar las predicciones para los nuevos textos
predictions_5 = []
x = 0
for i, text in enumerate(texts_to_classify):
    try:
        # Tokenizar el texto para obtener los token_type_ids
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        # Extraer la cadena de texto del tensor de entrada
        text_input = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
        # Hacer la predicción con la cadena de texto
        output_5 = sentiment_task(text_input)
        # Obtener la etiqueta de la predicción
        label = output_5[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'LABEL_2':
            predictions_5.append(1)
        elif label == 'LABEL_1':
            predictions_5.append(2)
        else:
            predictions_5.append(3)
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_5.append(3)
    x += 1
    print(f"Texto {x}: {output_5}")

print(predictions_5)


Texto 1: [{'label': 'LABEL_1', 'score': 0.5383895635604858}]
Texto 2: [{'label': 'LABEL_0', 'score': 0.867169201374054}]
Texto 3: [{'label': 'LABEL_2', 'score': 0.5790663957595825}]
Texto 4: [{'label': 'LABEL_0', 'score': 0.8389055132865906}]
Texto 5: [{'label': 'LABEL_1', 'score': 0.37894439697265625}]
Texto 6: [{'label': 'LABEL_0', 'score': 0.7470191121101379}]
Texto 7: [{'label': 'LABEL_2', 'score': 0.9179736971855164}]
Texto 8: [{'label': 'LABEL_1', 'score': 0.6306244134902954}]
Texto 9: [{'label': 'LABEL_0', 'score': 0.6754271984100342}]
Texto 10: [{'label': 'LABEL_1', 'score': 0.637685239315033}]
Texto 11: [{'label': 'LABEL_1', 'score': 0.8124867677688599}]
Texto 12: [{'label': 'LABEL_2', 'score': 0.7161735892295837}]
Texto 13: [{'label': 'LABEL_2', 'score': 0.7916954159736633}]
Texto 14: [{'label': 'LABEL_1', 'score': 0.5530030131340027}]
Texto 15: [{'label': 'LABEL_0', 'score': 0.7148367166519165}]
Texto 16: [{'label': 'LABEL_0', 'score': 0.9143281579017639}]
Texto 17: [{'label

In [ ]:
# Crear un DataFrame con las predicciones
results_sentiment_clas = pd.DataFrame({'id': filtered_data['id'], 'keyword': filtered_data['keyword'], 'text': filtered_data['text'], 'label': predictions_5})

print(results_sentiment_clas)


          id                               keyword  \
19    9ujfy8      running , running , bike, riding   
65    ppk7o2                      nature, outside    
66   d9lzwq4  beach,  sand,  sand,  breeze,  waves   
69    rbybrj                         walk,  stream   
76    eqxslg                                 climb   
..       ...                                   ...   
595  edvs552                                  walk   
596   ee31pf                               outside   
597  eei3pz3                         outside, walk   
598  eek8bpk                               outside   
599  eeljxq0                               outside   

                                                  text  label  
19   Still, there were limits to what I was comfort...      2  
65   My heart breaks, I leave and outside walk alon...      3  
66   I try to feel the sand under my feet until I f...      1  
69   and then i fucking stops there with tears stre...      3  
76   My brother took me on a da

In [ ]:
# Calcular el conteo de cada clase
class_counts = {label: predictions_5.count(label) for label in set(predictions_5)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

Conteo de cada clase:
Clase 1: 60 instancias
Clase 2: 92 instancias
Clase 3: 85 instancias


In [ ]:
# Seleccionar las filas con etiqueta 1 en results_df_original_data_0_1
label_1_rows = results_df_original_data_0_1[results_df_original_data_0_1['label'] == 1]
print(label_1_rows)


          id                               keyword  \
19    9ujfy8      running , running , bike, riding   
65    ppk7o2                      nature, outside    
66   d9lzwq4  beach,  sand,  sand,  breeze,  waves   
69    rbybrj                         walk,  stream   
76    eqxslg                                 climb   
..       ...                                   ...   
595  edvs552                                  walk   
596   ee31pf                               outside   
597  eei3pz3                         outside, walk   
598  eek8bpk                               outside   
599  eeljxq0                               outside   

                                                  text  label  
19    I thought I'd overcome my social anxiety but ...      1  
65    My nightmares aren’t about monsters, they are...      1  
66    First I try visualization: I try visualising ...      1  
69    cried in in front of everyone in my college c...      1  
76   Booking a rock climbing le

In [ ]:
# Actualizar la columna label con los valores correspondientes de results_sentiment_clas
label_1_rows['label'] = results_sentiment_clas['label'].values
print(label_1_rows)

          id                               keyword  \
19    9ujfy8      running , running , bike, riding   
65    ppk7o2                      nature, outside    
66   d9lzwq4  beach,  sand,  sand,  breeze,  waves   
69    rbybrj                         walk,  stream   
76    eqxslg                                 climb   
..       ...                                   ...   
595  edvs552                                  walk   
596   ee31pf                               outside   
597  eei3pz3                         outside, walk   
598  eek8bpk                               outside   
599  eeljxq0                               outside   

                                                  text  label  
19    I thought I'd overcome my social anxiety but ...      2  
65    My nightmares aren’t about monsters, they are...      3  
66    First I try visualization: I try visualising ...      1  
69    cried in in front of everyone in my college c...      3  
76   Booking a rock climbing le

<ipython-input-84-79ec2353301f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_1_rows['label'] = results_sentiment_clas['label'].values


In [ ]:
# Iterar sobre las filas del DataFrame más pequeño y actualizar el DataFrame mayor
for index, row in label_1_rows.iterrows():
    results_df_original_data_0_1.loc[index] = row

# Mostrar el DataFrame actualizado
print(results_df_original_data_0_1)


          id              keyword  \
0    fc6l72u               cruise   
1     8eijpy  runners, run , run    
2    didnpbe       running , golf   
3    dxe6gbb                  sea   
4    f5jqgbz                waves   
..       ...                  ...   
595  edvs552                 walk   
596   ee31pf              outside   
597  eei3pz3        outside, walk   
598  eek8bpk              outside   
599  eeljxq0              outside   

                                                  text  label  
0    (1) I had SA but managed to practically eradic...      0  
1     Anyone looking for a friend? Real friend, beh...      0  
2     Breathe, darling. The purpose of dating is to...      0  
3     I've been overweight since 2nd grade and it a...      0  
4     I need this terribly. I’ve been in negative w...      0  
..                                                 ...    ...  
595   The thought of applying for a job terrified m...      1  
596   No one will ever love me because I do

In [ ]:
# Obtener las etiquetas originales del conjunto de datos
original_data_all_labels = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0, 1, 2, 3], engine='python')


In [ ]:
y_true = original_data_all_labels['label']
y_pred = results_df_original_data_0_1['label']
# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       377
           1       0.38      0.43      0.40        54
           2       0.65      0.46      0.54       131
           3       0.36      0.82      0.50        38

    accuracy                           0.79       600
   macro avg       0.60      0.66      0.60       600
weighted avg       0.82      0.79      0.79       600

